# Text Compression

In this example we will analyze text compression using Huffman code and other codes as well. We will use texts downloaded from the Gutenberg database: Ulysses by James Joyce and The Complete Work of William Shakespeare.

In [16]:
ulysses="/tmp/ulysses.txt"
[ -f $ulysses ] && echo "$ulysses exists" || wget -q http://www.gutenberg.org/files/4300/4300-0.txt -O $ulysses
shakespeare="/tmp/shakespeare.txt.gz"
[ -f shakespeare ] && echo "$shakespeare exists" || wget -q http://www.gutenberg.org/cache/epub/100/pg100.txt -O $shakespeare

/tmp/ulysses.txt exists


Lets check the type of files we have downloaded.

In [17]:
file $ulysses
file $shakespeare

/tmp/ulysses.txt: UTF-8 Unicode (with BOM) text, with CRLF line terminators
/tmp/shakespeare.txt.gz: gzip compressed data, was "pg100.txt.utf8.gzip", last modified: Sun Oct  1 05:16:45 2017, max compression, original size 5589889


Note that ulysses.txt is encoded in UTF-8 and using BOM (byte order mark), which is not necessary. It also uses Windows line breaks using two symbols CR (carriage return) and LB (line break). 

"The UTF-8 BOM is a sequence of bytes (EF BB BF) that allows the reader to identify a file as being encoded in UTF-8.
Normally, the BOM is used to signal the endianness of an encoding, but since endianness is irrelevant to UTF-8, the BOM is unnecessary.
According to the Unicode standard, the BOM for UTF-8 files is not recommended"
(https://stackoverflow.com/questions/2223882/whats-different-between-utf-8-and-utf-8-without-bom)

In [22]:
tail --bytes=+4 $ulysses | tr -d '\r' > ${ulysses%.txt}2.txt
mv ${ulysses%.txt}2.txt $ulysses

Let's check again the file type and see it is cleaner as desired.
We used **tail** to remove 16-bit code used as BOM and **tr** to remove cariage returns.

In [23]:
file $ulysses

/tmp/ulysses.txt: UTF-8 Unicode text


Now we have a UTF-8 coded text. Not that it is not ASCII. It means the file has some non ASCII characters. Note that UTF-8 is compatible with ASCII (that means ASCII is a subset of UTF-8).

Shakespeare file was compressed by gunzip. Let's decompress it and ckeck the text file.

In [18]:
gzip -d $shakespeare

In [21]:
shakespeare=${shakespeare%.gz}
file $shakespeare

/tmp/shakespeare.txt: UTF-8 Unicode (with BOM) text, with CRLF line terminators


It has the same encoding ulysses.txt had. So we might do the same steps we did with ulysses.txt.

In [24]:
tail --bytes=+4 $shakespeare | tr -d '\r' > ${shakespeare%.txt}2.txt
mv ${shakespeare%.txt}2.txt $shakespeare
file $shakespeare

/tmp/shakespeare.txt: ASCII text


We have a pure ASCII file.

Lets define a function to count the frequency of occurrence of characters. We are gonna convert the text to smallcaps and remove everything that is not [a-z\n ]. 

In [52]:
cat $ulysses | tr -dc 'A-Za-z\n ' | tr 'A-Z' 'a-z' | tr '\n' ' ' | 
   tr -s ' ' > tmp && mv tmp $ulysses
cat $shakespeare | tr -dc 'A-Za-z\n ' | tr 'A-Z' 'a-z' | tr '\n' ' ' | 
   tr -s ' ' > tmp && mv tmp $shakespeare

rm: cannot remove 'tmp': No such file or directory


: 1

In [53]:
charcount() {
    FILENAME="$1"
    cat $FILENAME | sed 's/\(.\)/\1\n/g' | 
       sort | uniq -c | sort -n -r > "${FILENAME}_charcount"
    cat "${FILENAME}_charcount" | tr -dc '0-9\n' > "${FILENAME}_ccount"
}

In [39]:
charcount $ulysses
charcount $shakespeare

Now, let's check the result generated by the script above.

In [33]:
cat "${ulysses}_charcount" | column 

 267159  	  81154 n	  33764 u	  24597 y	   1466 x
 143271 e	  77671 s	  31891 m	  22859 p	   1342 q
 101741 t	  73078 h	  30507 c	  21433 b	   1076 z
  94117 a	  71037 r	  28220 g	  12203 k
  92726 o	  55540 l	  27010 f	   9870 v
  82481 i	  49606 d	  26453 w	   2414 j


To compute the entropy using a GNU Octave script directly from Bash, we created the script entropyfromdata.m bellow.

In [35]:
cat entropyfromdata.m
chmod +x entropyfromdata.m

#!/usr/bin/octave -qf

arg_list = argv ();

if( length(arg_list) == 0), error('no input file was given!'); endif;
filename = arg_list{1};
if( exist(filename,'file') != 2 ), error('file does not exist!'); endif;
n = load(filename);
if( size(n,2) != 1), error('a row vector file must be provided!'); endif;
if( any(floor(n) != n) ), error('you must provide a vector of integers!'); endif


if length( find(n==0,1) > 0), 
   n += ones(size(n)); % add one smothing
endif

% mle
p = n./sum(n);
H = entropy(p);
printf('Entropy: %.2f bits\n',H);



Bellow we use the script above to compute the entropy (bits/char) of both text files.

In [36]:
./entropyfromdata.m "${ulysses}_ccount"

Entropy: 4.12 bits


In [37]:
./entropyfromdata.m "${shakespeare}_ccount"

Entropy: 4.09 bits


We have estimated an entropy of about 4.1 bits/char for English text. However, we know there is much interdependence between characters in a sequence, what was not grasped in previous estimation, where we have assumed a i.i.d. source of characters. 

To improve our entropy of English text estimation, we will consider now words as unities. We need to compute the word entropy and divite it by the average word length.

In [54]:
wordcount() {
    FILENAME="$1"
    cat $FILENAME | tr ' ' '\n' | sort | uniq -c | sort -n -r > "${FILENAME}_wordcount"
    cat "${FILENAME}_wordcount" | tr -dc '0-9\n' > "${FILENAME}_wcount"
    cat "${FILENAME}_wordcount" | sed -E "s/\s*([0-9]+)\s([a-z]+)$/\1,\2/" > "${FILENAME}_wcount.csv"
    echo "count,word" | cat - "${FILENAME}_wcount.csv" | sponge "${FILENAME}_wcount.csv"
    # remove possible empty words
    awk -F, -i inplace -v INPLACE_SUFFIX=.bak 'length($2) {print}' "${FILENAME}_wcount.csv"
}

In [43]:
wordcount $ulysses
wordcount $shakespeare

Check the resulting file to see its content.

In [44]:
head -n 25 "${ulysses}_wcount.csv" | column 

count,word	5040,to		2621,that	1963,for	1337,all
15115,the	4986,in		2562,with	1961,you	1301,at
8262,of		4034,he		2371,it		1785,her	1297,by
7285,and	3330,his	2135,was	1523,him	1208,said
6560,a		2687,i		2120,on		1458,is		1197,as


Now, lets compute the entropy (per word) of the text files.

In [45]:
./entropyfromdata.m "${ulysses}_wcount"

Entropy: 10.62 bits


In [46]:
./entropyfromdata.m "${shakespeare}_wcount"

Entropy: 10.00 bits


We need now to compute the average word length. It might be easily done with awk.

In [47]:
awk -F, '{WLEN+=length($2)} END{print WLEN/NR}' "${ulysses}_wcount.csv"

7.54308


In [48]:
awk -F, '{WLEN+=length($2)} END{print WLEN/NR}' "${shakespeare}_wcount.csv"

7.45483


We may now compute the entropy of the texts in bits/char.

Ulysses:
10.62 bits / 7.54308 chars = 1.41 bits/char

Shakespeare:
10.00 bits / 7.45483 chars = 1.34 bits/char


If we compute the number of characters in these files and use the estimate entropy, we might estimate the theorical compression limit for these files. 

In [55]:
wc -m $ulysses

1464686 /tmp/ulysses.txt


In [56]:
wc -m $shakespeare

4707386 /tmp/shakespeare.txt


We shall have the following compression limits:

* considering the character entropy for a source that produces i.i.d. characters 
  1. ulysses: 1464686*4.12 = 6034506 bits (754313 bytes);
  2. shakespeare: 4707386*4.09 = 19253209 bits (2406651 bytes)
* considering a source that produces words we have estimated a lower entropy per character
  1. ulysses: 1464686*1.41 = 2065207 bits (258151 bytes);
  2.  shakespeare: 4707386*1.34 = 6307897 bits (788487 bytes)


To compress the files using different methods, we shall use the Basic Compression Library, available at http://bcl.comli.eu/ (alternatively here: https://github.com/MariadeAnton/bcl).

We will use git to clone the repository locally and then compile it.

In [58]:
git clone https://github.com/MariadeAnton/bcl /tmp/bcl

Cloning into '/tmp/bcl'...
remote: Enumerating objects: 143, done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 143
Receiving objects: 100% (143/143), 338.96 KiB | 350.00 KiB/s, done.
Resolving deltas: 100% (46/46), done.


In [61]:
CURRENTFOLDER=$(pwd)
cd /tmp/bcl/src/
make
cd $CURRENTFOLDER

gcc -c -Wall -W -ansi -pedantic -O3 rle.c
gcc -c -Wall -W -ansi -pedantic -O3 shannonfano.c
gcc -c -Wall -W -ansi -pedantic -O3 huffman.c
gcc -c -Wall -W -ansi -pedantic -O3 rice.c
gcc -c -Wall -W -ansi -pedantic -O3 lz.c
ar -rcs libbcl.a rle.o shannonfano.o huffman.o rice.o lz.o
gcc -c -Wall -w -O3 -s bfc.c
gcc -s bfc.o libbcl.a -o bfc.exe
gcc -c -Wall -w -O3 -s bcltest.c
gcc -c -Wall -w -O3 -s systimer.c
gcc -s bcltest.o systimer.o libbcl.a -o bcltest.exe


Now that we have the executable available, lets compress the text files using Huffman and LZ.

In [65]:
/tmp/bcl/src/bfc.exe c huff $ulysses ${ulysses%txt}huf

Huffman compress /tmp/ulysses.txt to /tmp/ulysses.huf...
Input file: 1464686 bytes
Output file: 761928 bytes (52.0%)


In [66]:
/tmp/bcl/src/bfc.exe c huff $shakespeare ${shakespeare%txt}huf

Huffman compress /tmp/shakespeare.txt to /tmp/shakespeare.huf...
Input file: 4707386 bytes
Output file: 2434327 bytes (51.7%)


In [67]:
/tmp/bcl/src/bfc.exe c lz $ulysses ${ulysses%txt}lz

LZ77 compress /tmp/ulysses.txt to /tmp/ulysses.lz...
Input file: 1464686 bytes
Output file: 935970 bytes (63.9%)


In [68]:
/tmp/bcl/src/bfc.exe c lz $shakespeare ${shakespeare%txt}lz

LZ77 compress /tmp/shakespeare.txt to /tmp/shakespeare.lz...
Input file: 4707386 bytes
Output file: 2779499 bytes (59.0%)


The table bellow presents the results achieved. 

obs.: the theoretical limits are for a source of characters model (c) or a source of words model (w).


| compression           | ulysses     | shakespeare |
|-----------------------|-------------|-------------|
| no compression        |1464686 bytes|4707386 bytes|
| theoretical limit (c) | 754313 bytes|2406651 bytes|
| theoretical limit (w) | 258151 bytes| 788487 bytes|
| Huffman               | 761928 bytes|2434327 bytes|
| eficiency_huffman (c) |  0.990      |  0.989      |
| eficiency_huffman (w) |  0.338      |  0.323      |
| LZ                    | 935970 bytes|2779499 bytes|
| eficiency_lz (c)      |  0.806      |  0.866      |
| eficiency_lz (w)      |  0.276      |  0.284      |

## GNU Octave

Lets now use Huffman code in GNU Octave to compress the text file.

Bellow is the code of a function that might be run from bash.

In [75]:
cat huffmancompress.m

#!/usr/bin/octave -qf
pkg load communications
pkg load image

% check input arguments
arg_list = argv ();
if( length(arg_list) == 0), error('no input file was given!'); endif;
for i=1:length(arg_list),
  if( exist(arg_list{i},'file') != 2 ), error('file does not exist!'); endif;
end;
% get filename
if length(arg_list) == 1,
  filename = arg_list{1};
else error('too many inputs!');
endif;

text = fileread (filename);
symbols = unique ( sort (text) );
counts = arrayfun (@(x) sum(ismember(text,x)),symbols);
[counts, id] = sort (counts,'descend');  
symbols = symbols(id);

p = counts./sum(counts); % mle of the distribution
dict = huffmandict(symbols, p);
% the function huffmanenco requeries integer index as input
% therefore we will create a map to convert chars into integers
% the mapped text will be used in the encoding method
symbolmap = zeros(1,255);
s2a = double (symbols); % convert to ascii values
symbolmap(s2a([1:length(symbols)])) = [1:length(symbols)];
out = huffmanenco(symbolmap(

In [76]:
chmod +x huffmancompress.m 
./huffmancompress.m $ulysses
ls -l "$ulysses.huf"

-rw-r--r-- 1 leoca leoca 764529 out 11 12:40 /tmp/ulysses.txt.huf


The file size is about the same as the file produced by the basic compression library. 

Decoding is performed by the function bellow:

In [77]:
cat huffmandecompress.m

#!/usr/bin/octave -qf
pkg load communications
pkg load image

% check input arguments
arg_list = argv ();
if( length(arg_list) == 0), error('no input file was given!'); endif;
for i=1:length(arg_list),
  if( exist(arg_list{i},'file') != 2 ), error('file does not exist!'); endif;
end;
% get filename
if length(arg_list) == 1,
  filename = arg_list{1};
else error('too many inputs!');
endif;

cmd = cstrcat("SPLITPOS=$(grep -abo '###END_HD###' ",filename," | cut -d: -f1) && head -c $SPLITPOS ",filename," > /tmp/huffdic && tail -c +`expr $SPLITPOS + 13` ",filename," > /tmp/data");
system (cmd);
load ('/tmp/huffdic');

fid = fopen ('/tmp/data', 'r');
x = fread (fid, Inf, 'uint8');
fclose (fid);

xbin = de2bi(x,8);
xbin = xbin(:);
back  = huffmandeco (xbin, dict);
id = find (back == -1);
back(id)=[];
decode = char (symbols(back));

fid = fopen(cstrcat(filename,'_decoded'),'w');
fwrite (fid, decode, 'char');
fclose(fid);


*obs: still not working properly 